In [1]:
import sqlalchemy as sqlalchemy_package
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import relationship
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.declarative import DeferredReflection
from sqlalchemy import *
import pandas as pd
import seaborn as sns
import warnings
import config as creds
from openpyxl import load_workbook
import pickle

sns.set()
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 40)
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
# Set up a connection to the postgres server
DATABASE_URL = f"postgres://{creds.PGUSER}:{creds.PGPASSWORD}@{creds.PGHOST}:5433/{creds.PGDATABASE}"
engine = sqlalchemy_package.create_engine(DATABASE_URL)

## Define Schema and Tables for Global Usage Table

In [3]:
# Set up a connection to the postgres server
class postgre_sql():
    def __init__(self, dbase = creds.PGDATABASE, dbschema = 'countries'):
        DATABASE_URL = f"postgres://{creds.PGUSER}:{creds.PGPASSWORD}@{creds.PGHOST}:5433/{dbase}"
        engine = sqlalchemy_package.create_engine(DATABASE_URL,
                connect_args={'options': '-csearch_path={}'.format(dbschema)})
        self.engine = engine
    def declare(self):
        # Declare a Base with explicit schema using `automap_base()`
        Base = declarative_base()
        return Base

In [4]:
# Creating objects
countries_db = postgre_sql()
base_db = countries_db.declare()

## Clean, Transform and Normalize Oil Data from BP

In [5]:
# Open the excel file using openpyxl

filepath = (r"..\resources\bp-stats-review-2020-all-data.xlsx")
wb = load_workbook(filename = filepath, read_only=True)
ws_prod = wb["Oil Production - Barrels"]
ws_cons = wb["Oil Consumption - Barrels"]
df_prod = pd.DataFrame(ws_prod.values)
df_cons = pd.DataFrame(ws_cons.values)

### First get clean data for Consumption

In [6]:
# remove unneeded rows from top and bottom
df_cons_hf = df_cons.iloc[2:108,:56]
# Rename the columns
df_cons_hf.columns = df_cons_hf.iloc[0]
df_cons_named = df_cons_hf.rename(columns={'Thousand barrels daily': 'country'})
#Take out top row again
df_cons_named1 = df_cons_named.iloc[1:]

# Remove the 'none' rows
df_removenone = df_cons_named1.loc[~df_cons_named["country"].str.contains("None", na = True)]

# Remove the 'total' rows
df_removetotal = df_removenone.loc[~df_removenone["country"].str.contains("Total", na = True)].reset_index(drop=True)

df_removetotal

2,country,1965,1966,1967,1968,1969,1970,1971,1972,1973,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Canada,"1,108.12","1,167.47","1,245.56","1,322.16","1,380.19","1,471.58","1,512.13","1,588.91","1,682.38",...,"2,332.64","2,396.55","2,329.52","2,345.34","2,386.79","2,346.75","2,393.33","2,393.01","2,443.31","2,402.54"
1,Mexico,315.87,333.05,357.31,386.14,410.02,440.74,467.28,523.35,564.18,...,"2,039.74","2,065.14","2,083.00","2,034.16","1,959.69","1,939.29","1,950.18","1,883.23","1,821.23","1,733.35"
2,US,"11,522.19","12,100.35","12,566.88","13,404.55","14,152.99","14,709.91","15,222.76","16,380.81","17,317.93",...,"18,324.38","17,983.22","17,593.72","18,005.84","18,136.22","18,523.68","18,617.64","18,883.34","19,427.64","19,399.96"
3,Argentina,432.37,447.27,459.88,468.99,491.75,447.79,479.60,478.58,482.63,...,581.81,591.29,613.89,657.63,642.84,662.42,650.51,636.87,611.04,599.06
4,Brazil,305.67,334.60,344.54,412.43,456.56,512.72,564.49,648.89,797.46,...,"2,271.41","2,431.73","2,509.39","2,655.47","2,721.19","2,559.26","2,436.21","2,480.68","2,376.58","2,397.53"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Sri Lanka,9.35,9.58,9.81,10.01,10.27,20.60,18.92,20.45,21.27,...,87.03,91.69,95.39,82.03,70.59,89.56,105.18,113.62,114.05,122.62
88,Taiwan,43.86,51.78,61.37,72.86,87.34,104.73,145.16,156.20,199.93,...,"1,043.41",950.45,949.61,981.14,"1,012.64","1,021.24","1,046.14","1,038.45","1,049.58",998.01
89,Thailand,47.56,55.99,61.97,81.13,87.97,102.81,114.39,142.75,151.43,...,"1,104.16","1,166.11","1,224.96","1,266.64","1,270.26","1,329.66","1,364.38","1,407.14","1,432.28","1,453.29"
90,Vietnam,31.35,68.51,100.34,102.08,123.71,128.26,108.50,111.72,109.93,...,332.29,361.38,371.44,397.81,409.08,452.12,470.15,487.39,506.39,528.46


### Repeat for Production

In [7]:
# remove unneeded rows from top and bottom
df_prod_hf = df_prod.iloc[2:72,:56]
# Rename the columns
df_prod_hf.columns = df_prod_hf.iloc[0]
df_prod_named = df_prod_hf.rename(columns={'Thousand barrels daily': 'country'})
#Take out top row again
df_prod_named1 = df_prod_named.iloc[1:]

# Remove the 'none' rows
df_removenone_prod = df_prod_named1.loc[~df_prod_named["country"].str.contains("None", na = True)]

# Remove the 'total' rows
df_removetotal_prod = df_removenone_prod.loc[~df_removenone_prod["country"].str.contains("Total", na = True)].reset_index(drop=True)


## Attempt to normalize the country names

In [8]:
# Transform Countries Data from CSV
countries_df = pd.read_csv('../countries/data/en/countries.csv')
countries =[]
for result in countries_df.itertuples():
    row = {}
    row['country_code'] = result.alpha3.upper()
    row['country'] = result.name
    countries.append(row)
print('List of official names of 206 world countries:\n', [row['country'] for row in countries])

List of official names of 206 world countries:
 ['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia (Plurinational State of)', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo', 'Congo, Democratic Republic of the', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Hond

In [9]:
not_in_list = []
for r in df_removetotal.itertuples():
    if (r[1] not in [row['country'] for row in countries]) and (r[1] not in not_in_list):
        not_in_list.append(r[1])
print('Number of inconsistent records: ', len(not_in_list))
print('\nList of records, which are not consistent with the country list:\n', not_in_list)

Number of inconsistent records:  23

List of records, which are not consistent with the country list:
 ['US', 'Trinidad & Tobago', 'Venezuela', 'Central America', 'Other Caribbean', 'Other South America', 'Czech Republic', 'United Kingdom', 'Other Europe ', 'USSR', 'Other CIS', 'Iran', 'Other Middle East', 'Eastern Africa', 'Middle Africa', 'Western Africa', 'Other Northern Africa', 'Other Southern Africa', 'China Hong Kong SAR', 'South Korea', 'Taiwan', 'Vietnam', 'Other Asia Pacific']


In [10]:
# Import list of countries with codes
df_countrycode = pd.read_csv("../outputs/data_countries.csv")
countries_clean =[]
for result in df_countrycode.itertuples():
    row = {}
    row['country_code'] = result.country_code
    row['country'] = result.country
    countries_clean.append(row)
print('List of country names for code lookup:\n', [row['country'] for row in countries])

List of country names for code lookup:
 ['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia (Plurinational State of)', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo', 'Congo, Democratic Republic of the', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', '

In [11]:
not_in_list2 = []
for r in df_removetotal.itertuples():
    if (r[1] not in [row['country'] for row in countries_clean]) and (r[1] not in not_in_list):
        not_in_list2.append(r[1])
print('Number of inconsistent records: ', len(not_in_list))
print('\nList of records, which are not consistent with the country list:\n', not_in_list)

Number of inconsistent records:  23

List of records, which are not consistent with the country list:
 ['US', 'Trinidad & Tobago', 'Venezuela', 'Central America', 'Other Caribbean', 'Other South America', 'Czech Republic', 'United Kingdom', 'Other Europe ', 'USSR', 'Other CIS', 'Iran', 'Other Middle East', 'Eastern Africa', 'Middle Africa', 'Western Africa', 'Other Northern Africa', 'Other Southern Africa', 'China Hong Kong SAR', 'South Korea', 'Taiwan', 'Vietnam', 'Other Asia Pacific']


In [12]:
remove_list = []
rename_dict = {'Aruba': 'Netherlands', 'Bermuda': 'United Kingdom', 'Cote d?Ivoire': 'Ivory Coast', 'Congo-Kinshasa': 'Democratic Republic of the Congo', 'Republic of Congo': 'Democratic Republic of the Congo',
               'Congo-Brazzaville': 'Republic of Congo', 'Cook Islands': 'New Zealand', 'Cabo Verde':'Guinea Bissau', 'Gibraltar':'United Kingdom',
               'Guadeloupe':'France', 'Guinea-Bissau':'Guinea Bissau', 'French Guiana':'France', 'Hawaiian Trade Zone': 'United States of America',
               'Hong Kong':'Hong Kong S.A.R.', 'Macau':'China', 'Maldives':'United Kingdom', 'North Macedonia':'Macedonia', 'Burma':'Myanmar',
               'Montserrat':'United Kingdom', 'Martinique':'France', 'Netherlands Antilles':'Netherlands', 'Nauru':'Australia', 
               'Palestinian Territories':'Palestine','Reunion':'France', 'Serbia':'Republic of Serbia', 'Eswatini':'Swaziland',
               'Seychelles':'United Kingdom', 'Turks and Caicos Islands':'United Kingdom', 'Timor-Leste':'United Kingdom',
               'Tanzania':'United Republic of Tanzania', 'U.S. Virgin Islands':'United States Virgin Islands', 'United States':'United States of America', 'US': 'United States of America',
               'U.S. Pacific Islands':'United States of America', 'Saint Vincent/Grenadines':'Saint Vincent and the Grenadines', 'British Virgin Islands':'United Kingdom',
               'Wake Island':'United States of America', 'Trinidad & Tobago': 'Trinidad and Tobago', 'Russian Federation': 'Russia', 'China Hong Kong SAR': 'Hong Kong S.A.R.',
               'USSR': 'Russia'}
norm_countries = pickle.load(open('../outputs/norm_countries.pkl', 'rb'))

In [13]:
# new_df = pd.DataFrame(columns = df_removetotal.columns)
# new_df

# newrow = pd.DataFrame(df_removetotal.loc[1])
# # newrow

# pd.merge(new_df, newrow, how="outer").merge()

# newrow2 = pd.DataFrame(df_removetotal.loc[2:3])
# newrow2

In [14]:
# Cleaning records table using dataframes instead of lists
def clean_dataframe(dataframe, rename_dict=rename_dict):
    new_df = pd.DataFrame(columns = dataframe.columns)
    for row in dataframe.itertuples(index=False):
        country = row[0]
        if country in rename_dict.keys():
            country = rename_dict[row[0]]
            newrow = pd.DataFrame([(country, row[1], row[2], row[3], row[4], row[5], row[6],
                                    row[7], row[8], row[9], row[10], row[11], row[12], row[13], row[14],
                                    row[15], row[16], row[17], row[18], row[19], row[20], row[21], row[22],
                                    row[23], row[24], row[25], row[26], row[27], row[28], row[29], row[30],
                                    row[31], row[32], row[33], row[34], row[35], row[36], row[37], row[38], row[39],
                                    row[40], row[41], row[42], row[43], row[44], row[45], row[46], row[47],
                                    row[48], row[49], row[50], row[51], row[52], row[53], row[54],row[55])], columns = dataframe.columns)
        else:
            newrow = pd.DataFrame([row], columns = dataframe.columns)
        new_df = pd.concat([new_df, newrow])
    return new_df

# Normalize Data Country_Code, using dataframes instead of lists
def norm_dataframe(data_list):
    new_df = pd.DataFrame(columns = dataframe.columns)
    for record in dataframe.itertuples(index=False):
        country = row[0]
        for row in norm_countries:
            if record['code'] == row['country']:
                record['code'] = row['country_code']
    return data_list

In [15]:
df_cons_clean = clean_dataframe(df_removetotal).reset_index(drop = True)
df_prod_clean = clean_dataframe(df_removetotal_prod).reset_index(drop = True)

df_prod_clean

2,country,1965,1966,1967,1968,1969,1970,1971,1972,1973,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Canada,920,"1,012.00",1106,1194,1306,1473,1582,1829,"2,114.00",...,"3,456.81","3,636.74","3,859.87","4,127.94","4,418.06","4,572.27","4,655.48","5,037.33","5,500.68","5,650.77"
1,Mexico,362.00,370.00,411,439,461.00,487,486,506,525.00,...,"2,959.37","2,940.31","2,911.10","2,874.75","2,784.20","2,586.54","2,455.80","2,224.15","2,068.37","1,918.14"
2,United States of America,"9,014.15","9,579.24","10,219.25","10,599.74","10,827.67","11,296.85","11,155.78","11,184.90","10,945.95",...,"7,558.37","7,882.73","8,926.46","10,098.68","11,801.45","12,781.00","12,348.54","13,134.85","15,359.94","17,044.60"
3,Argentina,275.67,292.86,318.63,348.16,361.53,399.46,431.54,444.12,433.64,...,711.84,667.24,656.66,644.37,637.59,646.36,609.63,590.06,591.32,620.41
4,Brazil,96.05,117.05,147.05,161.05,176.05,166.68,174.85,171.19,173.99,...,"2,125.05","2,173.05","2,131.58","2,095.98","2,341.38","2,524.98","2,591.35","2,720.90","2,678.89","2,876.53"
5,Colombia,203,199.00,192,176.00,214,226,224,203,192.00,...,785.86,915.26,944.12,"1,009.87",990.38,"1,005.57",885.90,854.04,865.48,885.83
6,Ecuador,8.00,7.00,6.00,5,4.00,4.00,4.00,78,209.00,...,488.09,500.61,504.62,527.35,556.55,543.09,548.39,531.31,517.24,531.00
7,Peru,66,66.00,74,77,75.00,74.97,63.88,66.60,72.59,...,165.21,158.51,157.23,171.32,175.23,152.95,140.70,136.64,153.63,142.50
8,Trinidad and Tobago,135,153.00,179,191.00,157,140,129,141,166.00,...,145.37,135.87,116.95,115.60,114.12,108.75,96.66,98.69,87.32,82.22
9,Venezuela,3503,"3,402.00",3576,3639,3631,3754,3615,"3,301.00","3,455.00",...,"2,841.57","2,755.02","2,703.73","2,680.36","2,692.49","2,630.86","2,347.00","2,096.04","1,474.70",918.26


## Add the country code to our dataframes

In [16]:
dfcons_final = pd.merge(df_cons_clean, df_countrycode[['country','country_code']], how='left', on = "country", copy = False )
dfprod_final = pd.merge(df_prod_clean, df_countrycode[['country','country_code']], how='left', on = "country", copy = False )
dfprod_final

,country,1965,1966,1967,1968,1969,1970,1971,1972,1973,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,country_code
0,Canada,920,"1,012.00",1106,1194,1306,1473,1582,1829,"2,114.00",...,"3,636.74","3,859.87","4,127.94","4,418.06","4,572.27","4,655.48","5,037.33","5,500.68","5,650.77",CAN
1,Mexico,362.00,370.00,411,439,461.00,487,486,506,525.00,...,"2,940.31","2,911.10","2,874.75","2,784.20","2,586.54","2,455.80","2,224.15","2,068.37","1,918.14",MEX
2,United States of America,"9,014.15","9,579.24","10,219.25","10,599.74","10,827.67","11,296.85","11,155.78","11,184.90","10,945.95",...,"7,882.73","8,926.46","10,098.68","11,801.45","12,781.00","12,348.54","13,134.85","15,359.94","17,044.60",USA
3,Argentina,275.67,292.86,318.63,348.16,361.53,399.46,431.54,444.12,433.64,...,667.24,656.66,644.37,637.59,646.36,609.63,590.06,591.32,620.41,ARG
4,Brazil,96.05,117.05,147.05,161.05,176.05,166.68,174.85,171.19,173.99,...,"2,173.05","2,131.58","2,095.98","2,341.38","2,524.98","2,591.35","2,720.90","2,678.89","2,876.53",BRA
5,Colombia,203,199.00,192,176.00,214,226,224,203,192.00,...,915.26,944.12,"1,009.87",990.38,"1,005.57",885.90,854.04,865.48,885.83,COL
6,Ecuador,8.00,7.00,6.00,5,4.00,4.00,4.00,78,209.00,...,500.61,504.62,527.35,556.55,543.09,548.39,531.31,517.24,531.00,ECU
7,Peru,66,66.00,74,77,75.00,74.97,63.88,66.60,72.59,...,158.51,157.23,171.32,175.23,152.95,140.70,136.64,153.63,142.50,PER
8,Trinidad and Tobago,135,153.00,179,191.00,157,140,129,141,166.00,...,135.87,116.95,115.60,114.12,108.75,96.66,98.69,87.32,82.22,TTO
9,Venezuela,3503,"3,402.00",3576,3639,3631,3754,3615,"3,301.00","3,455.00",...,"2,755.02","2,703.73","2,680.36","2,692.49","2,630.86","2,347.00","2,096.04","1,474.70",918.26,VEN


## Try to flatten the dataframe so each year is it's own row

In [18]:
dfcons_finalmelted = dfcons_final.melt(["country_code","country"], var_name = "year", value_name = "consumption")
dfprod_finalmelted = dfprod_final.melt(["country_code","country"], var_name = "year", value_name = "production")

## Write the dataframe to our Postgres Database

In [19]:
engine = countries_db.engine
dfcons_finalmelted.to_sql('bp_national_consumption',engine, if_exists='replace')
dfprod_finalmelted.to_sql('bp_national_production',engine, if_exists='replace')